In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/google-cloud-ncaa-march-madness-2020-division-1-mens-tournament/MEvents2016.csv
/kaggle/input/google-cloud-ncaa-march-madness-2020-division-1-mens-tournament/MEvents2018.csv
/kaggle/input/google-cloud-ncaa-march-madness-2020-division-1-mens-tournament/MEvents2015.csv
/kaggle/input/google-cloud-ncaa-march-madness-2020-division-1-mens-tournament/MEvents2019.csv
/kaggle/input/google-cloud-ncaa-march-madness-2020-division-1-mens-tournament/MEvents2017.csv
/kaggle/input/google-cloud-ncaa-march-madness-2020-division-1-mens-tournament/MSampleSubmissionStage1_2020.csv
/kaggle/input/google-cloud-ncaa-march-madness-2020-division-1-mens-tournament/MPlayers.csv
/kaggle/input/google-cloud-ncaa-march-madness-2020-division-1-mens-tournament/MDataFiles_Stage1/Cities.csv
/kaggle/input/google-cloud-ncaa-march-madness-2020-division-1-mens-tournament/MDataFiles_Stage1/MTeamCoaches.csv
/kaggle/input/google-cloud-ncaa-march-madness-2020-division-1-mens-tournament/MDataFiles_Stage1/MTeamSpellin

In [2]:
import pandas as pd
import numpy as np
import os
import sys

import re
import random
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold
import lightgbm as lgb
import xgboost as xgb
from xgboost import XGBClassifier
import gc
import glob

%matplotlib inline

In [3]:
df = {}
for i in glob.glob('/kaggle/input/google-cloud-ncaa-march-madness-2020-division-1-mens-tournament/MDataFiles_Stage1/*'):
    name = i.split('/')[-1].split('.')[0]
    if name != 'MTeamSpellings':
        df[name] = pd.read_csv(i)
    else:
        df[name] = pd.read_csv(i, encoding='cp1252')

In [4]:
df.keys()

dict_keys(['Cities', 'MTeamCoaches', 'MTeamSpellings', 'MMasseyOrdinals', 'MSeasons', 'MTeams', 'MSecondaryTourneyTeams', 'MNCAATourneyCompactResults', 'MGameCities', 'Conferences', 'MNCAATourneySeeds', 'MNCAATourneySlots', 'MTeamConferences', 'MNCAATourneySeedRoundSlots', 'MNCAATourneyDetailedResults', 'MConferenceTourneyGames', 'MRegularSeasonDetailedResults', 'MRegularSeasonCompactResults', 'MSecondaryTourneyCompactResults'])

In [5]:
regular=df['MRegularSeasonDetailedResults']

regular['WFGM2']=regular['WFGM']-regular['WFGM3']
regular['LFGM2']=regular['LFGM']-regular['LFGM3']

regular['WFGA2']=regular['WFGA']-regular['WFGA3']
regular['LFGA2']=regular['LFGA']-regular['LFGA3']

regular=regular[['Season','WTeamID','LTeamID','WScore','LScore','WLoc',
                 'WFGM', 'WFGA','WFGM3', 'WFGA3', 'WFGM2','WFGA2',
                 'LFGM', 'LFGA', 'LFGM3', 'LFGA3','LFGM2','LFGA2']]
regular

,Season,WTeamID,LTeamID,WScore,LScore,WLoc,WFGM,WFGA,WFGM3,WFGA3,WFGM2,WFGA2,LFGM,LFGA,LFGM3,LFGA3,LFGM2,LFGA2
0,2003,1104,1328,68,62,N,27,58,3,14,24,44,22,53,2,10,20,43
1,2003,1272,1393,70,63,N,26,62,8,20,18,42,24,67,6,24,18,43
2,2003,1266,1437,73,61,N,24,58,8,18,16,40,22,73,3,26,19,47
3,2003,1296,1457,56,50,N,18,38,3,9,15,29,18,49,6,22,12,27
4,2003,1400,1208,77,71,N,30,61,6,14,24,47,24,62,6,16,18,46
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87499,2019,1153,1222,69,57,N,22,50,6,17,16,33,19,62,8,33,11,29
87500,2019,1209,1426,73,64,N,20,50,8,22,12,28,23,64,7,33,16,31
87501,2019,1277,1276,65,60,N,22,55,9,23,13,32,21,51,8,25,13,26
87502,2019,1387,1382,55,53,N,22,59,4,22,18,37,19,56,7,19,12,37


In [6]:
tourney=df['MNCAATourneyDetailedResults']
tourney['WFGM2']=tourney['WFGM']-tourney['WFGM3']
tourney['LFGM2']=tourney['LFGM']-tourney['LFGM3']

tourney['WFGA2']=tourney['WFGA']-tourney['WFGA3']
tourney['LFGA2']=tourney['LFGA']-tourney['LFGA3']

tourney=tourney[['Season','WTeamID','LTeamID','WScore','LScore','WLoc',
                 'WFGM', 'WFGA','WFGM3', 'WFGA3', 'WFGM2','WFGA2',
                 'LFGM', 'LFGA', 'LFGM3', 'LFGA3','LFGM2','LFGA2']]
tourney

,Season,WTeamID,LTeamID,WScore,LScore,WLoc,WFGM,WFGA,WFGM3,WFGA3,WFGM2,WFGA2,LFGM,LFGA,LFGM3,LFGA3,LFGM2,LFGA2
0,2003,1421,1411,92,84,N,32,69,11,29,21,40,29,67,12,31,17,36
1,2003,1112,1436,80,51,N,31,66,7,23,24,43,20,64,4,16,16,48
2,2003,1113,1272,84,71,N,31,59,6,14,25,45,25,69,7,28,18,41
3,2003,1141,1166,79,73,N,29,53,3,7,26,46,27,60,7,17,20,43
4,2003,1143,1301,76,74,N,27,64,7,20,20,44,25,56,9,21,16,35
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1110,2019,1120,1246,77,71,N,26,65,7,23,19,42,27,61,5,21,22,40
1111,2019,1277,1181,68,67,N,30,70,6,19,24,51,26,57,7,21,19,36
1112,2019,1403,1277,61,51,N,22,51,9,23,13,28,15,47,7,24,8,23
1113,2019,1438,1120,63,62,N,25,51,7,19,18,32,21,55,9,31,12,24


In [7]:
rw=regular[['Season','WTeamID','WFGM','WFGA', 'WFGM3', 'WFGA3', 'WFGM2', 'WFGA2']]
rw=rw.groupby(['Season','WTeamID']).mean()

tw=tourney[['Season','WTeamID','WFGM','WFGA', 'WFGM3', 'WFGA3', 'WFGM2', 'WFGA2']]
tw=tw.groupby(['Season','WTeamID']).mean()

rtw=pd.concat((rw,tw))
rtw=rtw.groupby(['Season','WTeamID']).mean()
rtw

WFGM       WFGA      WFGM3      WFGA3      WFGM2  \
Season WTeamID                                                          
2003   1102     22.583333  40.000000  10.000000  21.583333  12.583333   
       1103     30.000000  55.384615   5.461538  14.384615  24.538462   
       1104     25.823529  58.352941   7.058824  20.823529  18.764706   
       1105     25.571429  61.857143   9.142857  22.428571  16.428571   
       1106     24.769231  53.846154   5.846154  15.923077  18.923077   
...                   ...        ...        ...        ...        ...   
2019   1462     27.666667  55.388889   7.555556  21.000000  20.111111   
       1463     31.000000  59.857143   8.000000  20.000000  23.000000   
       1464     28.000000  63.400000  10.100000  27.600000  17.900000   
       1465     27.416667  57.500000   9.583333  24.333333  17.833333   
       1466     24.857143  59.857143   8.000000  23.857143  16.857143   

                    WFGA2  
Season WTeamID             
2003   1102     18.416667  
       1103     41.000000  
       1104     37.529412  
       1105     39.428571  
       1106     37.923077  
...                   ...  
2019   1462     34.388889  
       1463     39.857143  
       1464     35.800000  
       1465     33.166667  
       1466     36.000000  

[5829 rows x 6 columns]

In [8]:
rl=regular[['Season','LTeamID','LFGM', 'LFGA', 'LFGM3','LFGA3', 'LFGM2', 'LFGA2']]
rl=rl.groupby(['Season','LTeamID']).mean()

tl=tourney[['Season','LTeamID','LFGM', 'LFGA', 'LFGM3','LFGA3', 'LFGM2', 'LFGA2']]
tl=tl.groupby(['Season','LTeamID']).mean()

rtl=pd.concat((rl,tl))
rtl=rtl.groupby(['Season','LTeamID']).mean()
rtl

LFGM       LFGA     LFGM3      LFGA3      LFGM2  \
Season LTeamID                                                         
2003   1102     16.562500  39.625000  6.187500  20.250000  10.375000   
       1103     24.500000  56.285714  5.428571  17.642857  19.071429   
       1104     21.636364  53.681818  5.136364  15.181818  16.500000   
       1105     23.947368  61.526316  7.000000  20.157895  16.947368   
       1106     22.266667  56.533333  6.333333  19.133333  15.933333   
...                   ...        ...       ...        ...        ...   
2019   1462     24.266667  56.400000  6.333333  21.066667  17.933333   
       1463     26.642857  66.428571  7.571429  30.142857  19.071429   
       1464     26.250000  63.750000  9.300000  28.200000  16.950000   
       1465     24.857143  60.357143  8.142857  26.000000  16.714286   
       1466     22.318182  59.590909  7.000000  24.500000  15.318182   

                    LFGA2  
Season LTeamID             
2003   1102     19.375000  
       1103     38.642857  
       1104     38.500000  
       1105     41.368421  
       1106     37.400000  
...                   ...  
2019   1462     35.333333  
       1463     36.285714  
       1464     35.550000  
       1465     34.357143  
       1466     35.090909  

[5834 rows x 6 columns]

In [9]:
secondary=df['MSecondaryTourneyCompactResults']

secondary=secondary[['Season','WTeamID','LTeamID','WScore', 'LScore','WLoc']]
regular=regular[['Season','WTeamID','LTeamID','WScore', 'LScore','WLoc']]
tourney=tourney[['Season','WTeamID','LTeamID','WScore', 'LScore','WLoc']]

all_match=pd.concat((regular,tourney,secondary))

all_match=pd.merge(all_match,rtw,how='left',left_on=['Season','WTeamID'],right_on=['Season','WTeamID'])
all_match=pd.merge(all_match,rtl,how='left',left_on=['Season','LTeamID'],right_on=['Season','LTeamID'])
all_match

,Season,WTeamID,LTeamID,WScore,LScore,WLoc,WFGM,WFGA,WFGM3,WFGA3,WFGM2,WFGA2,LFGM,LFGA,LFGM3,LFGA3,LFGM2,LFGA2
0,2003,1104,1328,68,62,N,25.823529,58.352941,7.058824,20.823529,18.764706,37.529412,18.916667,56.416667,5.166667,22.666667,13.750000,33.750000
1,2003,1272,1393,70,63,N,26.652174,59.434783,7.217391,19.956522,19.434783,39.478261,23.000000,63.800000,2.800000,16.000000,20.200000,47.800000
2,2003,1266,1437,73,61,N,28.195652,55.206522,7.429348,15.451087,20.766304,39.755435,23.333333,59.133333,5.800000,19.533333,17.533333,39.600000
3,2003,1296,1457,56,50,N,24.705882,51.352941,6.117647,14.941176,18.588235,36.411765,22.100000,59.900000,6.100000,22.200000,16.000000,37.700000
4,2003,1400,1208,77,71,N,27.102273,61.090909,5.500000,15.511364,21.602273,45.579545,27.125000,62.375000,5.625000,15.875000,21.500000,46.500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90238,2019,1453,1411,87,86,H,29.933333,62.733333,8.066667,21.733333,21.866667,41.000000,25.538462,65.923077,6.461538,25.153846,19.076923,40.769231
90239,2019,1177,1378,100,96,H,29.066667,58.866667,7.066667,19.600000,22.000000,39.266667,22.615385,56.615385,6.769231,23.538462,15.846154,33.076923
90240,2019,1267,1453,90,70,H,30.157895,62.789474,10.736842,27.894737,19.421053,34.894737,25.750000,62.187500,7.500000,24.000000,18.250000,38.187500
90241,2019,1400,1252,81,66,N,26.437500,56.562500,9.625000,25.312500,16.812500,31.250000,24.000000,61.142857,6.428571,20.857143,17.571429,40.285714


In [10]:
all_match.isna().sum()

Season       0
WTeamID      0
LTeamID      0
WScore       0
LScore       0
WLoc         0
WFGM       584
WFGA       584
WFGM3      584
WFGA3      584
WFGM2      584
WFGA2      584
LFGM       584
LFGA       584
LFGM3      584
LFGA3      584
LFGM2      584
LFGA2      584
dtype: int64

In [11]:
all_match=all_match.dropna()
all_match

,Season,WTeamID,LTeamID,WScore,LScore,WLoc,WFGM,WFGA,WFGM3,WFGA3,WFGM2,WFGA2,LFGM,LFGA,LFGM3,LFGA3,LFGM2,LFGA2
0,2003,1104,1328,68,62,N,25.823529,58.352941,7.058824,20.823529,18.764706,37.529412,18.916667,56.416667,5.166667,22.666667,13.750000,33.750000
1,2003,1272,1393,70,63,N,26.652174,59.434783,7.217391,19.956522,19.434783,39.478261,23.000000,63.800000,2.800000,16.000000,20.200000,47.800000
2,2003,1266,1437,73,61,N,28.195652,55.206522,7.429348,15.451087,20.766304,39.755435,23.333333,59.133333,5.800000,19.533333,17.533333,39.600000
3,2003,1296,1457,56,50,N,24.705882,51.352941,6.117647,14.941176,18.588235,36.411765,22.100000,59.900000,6.100000,22.200000,16.000000,37.700000
4,2003,1400,1208,77,71,N,27.102273,61.090909,5.500000,15.511364,21.602273,45.579545,27.125000,62.375000,5.625000,15.875000,21.500000,46.500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90238,2019,1453,1411,87,86,H,29.933333,62.733333,8.066667,21.733333,21.866667,41.000000,25.538462,65.923077,6.461538,25.153846,19.076923,40.769231
90239,2019,1177,1378,100,96,H,29.066667,58.866667,7.066667,19.600000,22.000000,39.266667,22.615385,56.615385,6.769231,23.538462,15.846154,33.076923
90240,2019,1267,1453,90,70,H,30.157895,62.789474,10.736842,27.894737,19.421053,34.894737,25.750000,62.187500,7.500000,24.000000,18.250000,38.187500
90241,2019,1400,1252,81,66,N,26.437500,56.562500,9.625000,25.312500,16.812500,31.250000,24.000000,61.142857,6.428571,20.857143,17.571429,40.285714


In [12]:

all_match.groupby('WLoc').size()


WLoc
A    27053
H    52511
N    10095
dtype: int64

In [13]:
seeds=df['MNCAATourneySeeds']
seeds

,Season,Seed,TeamID
0,1985,W01,1207
1,1985,W02,1210
2,1985,W03,1228
3,1985,W04,1260
4,1985,W05,1374
...,...,...,...
2281,2019,Z12,1332
2282,2019,Z13,1414
2283,2019,Z14,1330
2284,2019,Z15,1159


In [14]:
seeds['Seed'] = seeds['Seed'].apply(lambda x: int(x[1:3]))
seeds

,Season,Seed,TeamID
0,1985,1,1207
1,1985,2,1210
2,1985,3,1228
3,1985,4,1260
4,1985,5,1374
...,...,...,...
2281,2019,12,1332
2282,2019,13,1414
2283,2019,14,1330
2284,2019,15,1159


In [15]:
data=pd.merge(all_match,seeds,how='left',left_on=['Season','WTeamID'],right_on=['Season','TeamID'])
data.drop(['TeamID'],axis=1,inplace=True)
data.rename(columns={'Seed':'WSeed'},inplace=True)

data=pd.merge(data,seeds,how='left',left_on=['Season','LTeamID'],right_on=['Season','TeamID'])
data.drop(['TeamID'],axis=1,inplace=True)
data.rename(columns={'Seed':'LSeed'},inplace=True)

data=data[['Season', 'WTeamID', 'LTeamID', 'WSeed','LSeed', 'WScore', 'LScore', 'WLoc', 
           'WFGM', 'WFGA','WFGM3', 'WFGA3', 'WFGM2','WFGA2',
                 'LFGM', 'LFGA', 'LFGM3', 'LFGA3','LFGM2','LFGA2']]

                 
data

,Season,WTeamID,LTeamID,WSeed,LSeed,WScore,LScore,WLoc,WFGM,WFGA,WFGM3,WFGA3,WFGM2,WFGA2,LFGM,LFGA,LFGM3,LFGA3,LFGM2,LFGA2
0,2003,1104,1328,10.0,1.0,68,62,N,25.823529,58.352941,7.058824,20.823529,18.764706,37.529412,18.916667,56.416667,5.166667,22.666667,13.750000,33.750000
1,2003,1272,1393,7.0,3.0,70,63,N,26.652174,59.434783,7.217391,19.956522,19.434783,39.478261,23.000000,63.800000,2.800000,16.000000,20.200000,47.800000
2,2003,1266,1437,3.0,NaN,73,61,N,28.195652,55.206522,7.429348,15.451087,20.766304,39.755435,23.333333,59.133333,5.800000,19.533333,17.533333,39.600000
3,2003,1296,1457,NaN,NaN,56,50,N,24.705882,51.352941,6.117647,14.941176,18.588235,36.411765,22.100000,59.900000,6.100000,22.200000,16.000000,37.700000
4,2003,1400,1208,1.0,NaN,77,71,N,27.102273,61.090909,5.500000,15.511364,21.602273,45.579545,27.125000,62.375000,5.625000,15.875000,21.500000,46.500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89654,2019,1453,1411,NaN,NaN,87,86,H,29.933333,62.733333,8.066667,21.733333,21.866667,41.000000,25.538462,65.923077,6.461538,25.153846,19.076923,40.769231
89655,2019,1177,1378,NaN,NaN,100,96,H,29.066667,58.866667,7.066667,19.600000,22.000000,39.266667,22.615385,56.615385,6.769231,23.538462,15.846154,33.076923
89656,2019,1267,1453,NaN,NaN,90,70,H,30.157895,62.789474,10.736842,27.894737,19.421053,34.894737,25.750000,62.187500,7.500000,24.000000,18.250000,38.187500
89657,2019,1400,1252,NaN,NaN,81,66,N,26.437500,56.562500,9.625000,25.312500,16.812500,31.250000,24.000000,61.142857,6.428571,20.857143,17.571429,40.285714


In [16]:
data.isna().sum()
data=data.dropna()
data=data.reset_index(drop=True)

In [17]:
data.rename(columns={'WTeamID':'TeamA','LTeamID':'TeamB'},inplace=True)
data.rename(columns={'WScore':'AScore','LScore':'BScore'},inplace=True)
data.rename(columns={'WSeed':'ASeed','LSeed':'BSeed','WLoc':'Loc'},inplace=True)

data

,Season,TeamA,TeamB,ASeed,BSeed,AScore,BScore,Loc,WFGM,WFGA,WFGM3,WFGA3,WFGM2,WFGA2,LFGM,LFGA,LFGM3,LFGA3,LFGM2,LFGA2
0,2003,1104,1328,10.0,1.0,68,62,N,25.823529,58.352941,7.058824,20.823529,18.764706,37.529412,18.916667,56.416667,5.166667,22.666667,13.750000,33.750000
1,2003,1272,1393,7.0,3.0,70,63,N,26.652174,59.434783,7.217391,19.956522,19.434783,39.478261,23.000000,63.800000,2.800000,16.000000,20.200000,47.800000
2,2003,1323,1237,5.0,16.0,89,45,H,25.886364,58.522727,8.136364,19.727273,17.750000,38.795455,22.153846,57.769231,4.807692,16.307692,17.346154,41.461538
3,2003,1242,1221,2.0,14.0,81,57,H,32.465217,62.586957,4.878261,14.069565,27.586957,48.517391,22.125000,54.375000,5.000000,16.875000,17.125000,37.500000
4,2003,1390,1462,4.0,3.0,63,62,H,27.760870,60.826087,7.695652,21.434783,20.065217,39.391304,20.400000,56.800000,6.800000,19.400000,13.600000,37.400000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6067,2019,1120,1246,5.0,2.0,77,71,N,29.000000,61.525000,12.085000,30.365000,16.915000,31.160000,26.000000,59.333333,4.916667,19.166667,21.083333,40.166667
6068,2019,1277,1181,2.0,1.0,68,67,N,28.125000,58.714286,7.875000,21.535714,20.250000,37.178571,26.700000,63.200000,7.200000,25.300000,19.500000,37.900000
6069,2019,1403,1277,3.0,2.0,61,51,N,26.138462,54.892308,7.326923,19.665385,18.811538,35.226923,20.250000,50.916667,7.750000,22.333333,12.500000,28.583333
6070,2019,1438,1120,1.0,5.0,63,62,N,25.626437,55.123563,8.528736,22.810345,17.097701,32.313218,22.666667,57.222222,9.500000,29.777778,13.166667,27.444444


In [18]:
win=data.copy()
lose=data.copy()

lose['TeamA']=win['TeamB']
lose['TeamB']=win['TeamA']

lose['ASeed']=win['BSeed']
lose['BSeed']=win['ASeed']

lose['AScore']=win['BScore']
lose['BScore']=win['AScore']

lose['Loc']=lose['Loc'].apply(lambda x: 'H' if x=='N' else 'N' if x=='H' else 'A')

win['result']=1
lose['result']=0

total=pd.concat((win,lose))
total['Loc']=total['Loc'].apply(lambda x: '1' if x=='H' else '0' if x=='N' else '-1')

total['Seed_diff']=total['ASeed']-total['BSeed']
total['Score_diff']=total['AScore']-total['BScore']



Asum=total.groupby(['Season', 'TeamA'])['AScore'].sum().reset_index()
Asum.rename(columns={'AScore':'A_Sum'},inplace=True)
total=pd.merge(total,Asum,how='left',left_on=['Season','TeamA'],right_on=['Season','TeamA'])

Bsum=total.groupby(['Season', 'TeamB'])['BScore'].sum().reset_index()
Bsum.rename(columns={'BScore':'B_Sum'},inplace=True)
total=pd.merge(total,Bsum,how='left',left_on=['Season','TeamB'],right_on=['Season','TeamB'])

total

,Season,TeamA,TeamB,ASeed,BSeed,AScore,BScore,Loc,WFGM,WFGA,...,LFGA,LFGM3,LFGA3,LFGM2,LFGA2,result,Seed_diff,Score_diff,A_Sum,B_Sum
0,2003,1104,1328,10.0,1.0,68,62,0,25.823529,58.352941,...,56.416667,5.166667,22.666667,13.750000,33.750000,1,9.0,6,758,1160
1,2003,1272,1393,7.0,3.0,70,63,0,26.652174,59.434783,...,63.800000,2.800000,16.000000,20.200000,47.800000,1,4.0,7,598,1106
2,2003,1323,1237,5.0,16.0,89,45,1,25.886364,58.522727,...,57.769231,4.807692,16.307692,17.346154,41.461538,1,-11.0,44,1083,235
3,2003,1242,1221,2.0,14.0,81,57,1,32.465217,62.586957,...,54.375000,5.000000,16.875000,17.125000,37.500000,1,-12.0,24,1678,209
4,2003,1390,1462,4.0,3.0,63,62,1,27.760870,60.826087,...,56.800000,6.800000,19.400000,13.600000,37.400000,1,1.0,1,937,761
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12139,2019,1246,1120,2.0,5.0,71,77,1,29.000000,61.525000,...,59.333333,4.916667,19.166667,21.083333,40.166667,0,-3.0,-6,1470,1462
12140,2019,1181,1277,1.0,2.0,67,68,1,28.125000,58.714286,...,63.200000,7.200000,25.300000,19.500000,37.900000,0,-1.0,-1,1753,1609
12141,2019,1277,1403,2.0,3.0,51,61,1,26.138462,54.892308,...,50.916667,7.750000,22.333333,12.500000,28.583333,0,-1.0,-10,1609,1252
12142,2019,1120,1438,5.0,1.0,62,63,1,25.626437,55.123563,...,57.222222,9.500000,29.777778,13.166667,27.444444,0,4.0,-1,1462,1296


**Train data**

In [19]:
train=total[['Season', 'TeamA', 'TeamB', 'ASeed', 'BSeed', 'AScore', 'BScore', 'Loc',
        'WFGM', 'WFGA','WFGM3', 'WFGA3', 'WFGM2','WFGA2',
                 'LFGM', 'LFGA', 'LFGM3', 'LFGA3','LFGM2','LFGA2', 'Seed_diff','Score_diff','A_Sum','B_Sum'
       ,'result']]
train


,Season,TeamA,TeamB,ASeed,BSeed,AScore,BScore,Loc,WFGM,WFGA,...,LFGA,LFGM3,LFGA3,LFGM2,LFGA2,Seed_diff,Score_diff,A_Sum,B_Sum,result
0,2003,1104,1328,10.0,1.0,68,62,0,25.823529,58.352941,...,56.416667,5.166667,22.666667,13.750000,33.750000,9.0,6,758,1160,1
1,2003,1272,1393,7.0,3.0,70,63,0,26.652174,59.434783,...,63.800000,2.800000,16.000000,20.200000,47.800000,4.0,7,598,1106,1
2,2003,1323,1237,5.0,16.0,89,45,1,25.886364,58.522727,...,57.769231,4.807692,16.307692,17.346154,41.461538,-11.0,44,1083,235,1
3,2003,1242,1221,2.0,14.0,81,57,1,32.465217,62.586957,...,54.375000,5.000000,16.875000,17.125000,37.500000,-12.0,24,1678,209,1
4,2003,1390,1462,4.0,3.0,63,62,1,27.760870,60.826087,...,56.800000,6.800000,19.400000,13.600000,37.400000,1.0,1,937,761,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12139,2019,1246,1120,2.0,5.0,71,77,1,29.000000,61.525000,...,59.333333,4.916667,19.166667,21.083333,40.166667,-3.0,-6,1470,1462,0
12140,2019,1181,1277,1.0,2.0,67,68,1,28.125000,58.714286,...,63.200000,7.200000,25.300000,19.500000,37.900000,-1.0,-1,1753,1609,0
12141,2019,1277,1403,2.0,3.0,51,61,1,26.138462,54.892308,...,50.916667,7.750000,22.333333,12.500000,28.583333,-1.0,-10,1609,1252,0
12142,2019,1120,1438,5.0,1.0,62,63,1,25.626437,55.123563,...,57.222222,9.500000,29.777778,13.166667,27.444444,4.0,-1,1462,1296,0


test data

In [20]:
test=pd.read_csv('/kaggle/input/google-cloud-ncaa-march-madness-2020-division-1-mens-tournament/MSampleSubmissionStage1_2020.csv')
test

,ID,Pred
0,2015_1107_1112,0.5
1,2015_1107_1116,0.5
2,2015_1107_1124,0.5
3,2015_1107_1125,0.5
4,2015_1107_1129,0.5
...,...,...
11385,2019_1449_1459,0.5
11386,2019_1449_1463,0.5
11387,2019_1458_1459,0.5
11388,2019_1458_1463,0.5


In [21]:
test[['Season','A','B']]=test['ID'].str.split('_',expand=True)
test['Season']=test['Season'].astype(int)
test['A']=test['A'].astype(int)
test['B']=test['B'].astype(int)
test.drop(['ID','Pred'],axis=1,inplace=True)
test

,Season,A,B
0,2015,1107,1112
1,2015,1107,1116
2,2015,1107,1124
3,2015,1107,1125
4,2015,1107,1129
...,...,...,...
11385,2019,1449,1459
11386,2019,1449,1463
11387,2019,1458,1459
11388,2019,1458,1463


In [22]:
test=pd.merge(test,seeds,how='left',left_on=['Season','A'],right_on=['Season','TeamID'])
test

,Season,A,B,Seed,TeamID
0,2015,1107,1112,14,1107
1,2015,1107,1116,14,1107
2,2015,1107,1124,14,1107
3,2015,1107,1125,14,1107
4,2015,1107,1129,14,1107
...,...,...,...,...,...
11385,2019,1449,1459,9,1449
11386,2019,1449,1463,9,1449
11387,2019,1458,1459,5,1458
11388,2019,1458,1463,5,1458
